# -----------------PROYECTO FINAL-----------------

## 1. IMPORTAR LIBRERIAS

In [1]:
from keras.src.saving.saving_api import load_model
import cv2
import mediapipe as mp
import numpy as np
import os
import tensorflow as tf
import sys

## 2. INICIALIZAR MEDIAPIPE

In [ ]:
# Importa el módulo de manos de MediaPipe para la detección de landmarks
mp_hands = mp.solutions.hands

# Inicializa el detector de manos con configuración específica:
hands = mp_hands.Hands(
    static_image_mode=False,  # Modo video (mejor para secuencias)
    max_num_hands=2,          # Máximo número de manos a detectar
    min_detection_confidence=0.5,  # Confianza mínima para detección inicial
    min_tracking_confidence=0.5,   # Confianza mínima para seguimiento continuo
    model_complexity=1        # Balance entre precisión y rendimiento (0=ligero, 1=medio, 2=completo)
)

# Utilidades para dibujar los landmarks y conexiones en la imagen
mp_draw = mp.solutions.drawing_utils

# Directorio donde se almacenarán los datos del dataset
dataset_dir = "dataset_9"

# Ruta donde se guardará/recuperará el modelo entrenado
model_path = "gesture_model.h5"

# Longitud de las secuencias temporales (número de frames por muestra)
sequence_length = 30

# Número total de landmarks (21 puntos por mano * 3 coordenadas (x,y,z) * 2 manos)
total_landmarks = 126

# Lista para almacenar los nombres de los gestos/clases
gestures = []

# Variables para normalización de datos (media y desviación estándar)
X_mean = None
X_std = None

## Funciones principales

In [ ]:
# Función de inicialización del sistema
def init_system():
    global gestures  # Accede a la variable global para almacenar los gestos
    
    # Crea el directorio principal del dataset si no existe
    # exist_ok=True evita errores si el directorio ya existe
    os.makedirs(dataset_dir, exist_ok=True)  
    
    # Obtiene la lista de gestos existentes del disco
    gestures = get_existing_gestures()  

# Función para obtener los gestos ya registrados
def get_existing_gestures():
    # Lista comprensiva que recorre todos los elementos en el directorio del dataset
    return [d for d in os.listdir(dataset_dir)  # Lista todos los archivos/carpetas
           # Verifica si es un directorio (y no un archivo)
           if os.path.isdir(os.path.join(dataset_dir, d))]  

## DETECCION DE MANO

In [ ]:
def detect_hands():
    # Muestra mensaje de inicio para el usuario
    print("\nIniciando detección de manos. Presiona 'ESC' para salir.")
    
    # Inicializa la captura de video desde la cámara predeterminada (índice 0)
    cap = cv2.VideoCapture(0)

    # Bucle principal de procesamiento de frames
    while True:
        # Lee un frame de la cámara
        ret, frame = cap.read()
        
        # Verifica si la lectura del frame fue exitosa
        if not ret:
            break  # Sale del bucle si hay error

        # Convierte el frame de BGR (OpenCV) a RGB (MediaPipe)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Procesa el frame con el modelo de detección de manos
        results = hands.process(rgb_frame)

        # Si se detectaron landmarks de manos
        if results.multi_hand_landmarks:
            # Itera sobre cada mano detectada
            for hand_landmarks in results.multi_hand_landmarks:
                # Dibuja los landmarks y conexiones en el frame
                mp_draw.draw_landmarks(
                    frame, 
                    hand_landmarks, 
                    mp_hands.HAND_CONNECTIONS  # Dibuja conexiones entre landmarks
                )

        # Muestra el frame procesado en una ventana
        cv2.imshow("Detección de Manos", frame)
        
        # Verifica si se presionó la tecla ESC (código 27)
        if cv2.waitKey(1) & 0xFF == 27:
            break  # Sale del bucle

    # Libera los recursos de la cámara
    cap.release()
    
    # Cierra todas las ventanas de OpenCV
    cv2.destroyAllWindows()

## RECOLLECION DE DATOS 

In [ ]:
def collect_data():
    global gestures  # Accede a la variable global de gestos
    
    # Solicita datos al usuario
    gesture = input("\nIngrese la palabra o letra para la cual desea recolectar datos: ").upper()
    num_sequences = int(input("Ingrese el número de secuencias a capturar (recomendado: 50): "))
    
    # Crea directorio para almacenar los datos
    save_dir = os.path.join(dataset_dir, gesture)
    os.makedirs(save_dir, exist_ok=True)  # Crea el directorio si no existe

    # Mensajes informativos
    print(f"\nRecolectando datos para el gesto '{gesture}'. Presiona 'ESC' para cancelar.")
    print("Mantenga la seña frente a la cámara...")
    
    # Inicializa captura de video
    cap = cv2.VideoCapture(0)
    sequence = []  # Almacena la secuencia actual
    counter = 0    # Contador de secuencias guardadas

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Procesamiento del frame
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            # Extrae landmarks de hasta 2 manos
            all_landmarks = []
            for hand in results.multi_hand_landmarks[:2]:  # Máximo 2 manos
                for lm in hand.landmark:
                    all_landmarks.extend([lm.x, lm.y, lm.z])  # Coordenadas normalizadas
            
            # Rellena con ceros si solo hay una mano detectada
            if len(results.multi_hand_landmarks) < 2:
                all_landmarks += [0.0] * 63  # 21 landmarks * 3 coordenadas
            
            sequence.append(all_landmarks)  # Agrega a la secuencia
            
            # Dibuja landmarks en el frame
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Guarda la secuencia cuando alcanza la longitud deseada
        if len(sequence) == sequence_length:
            np.save(os.path.join(save_dir, f"secuencia_{counter}.npy"), sequence)
            counter += 1
            sequence = []  # Reinicia la secuencia
            print(f"Secuencias capturadas: {counter}/{num_sequences}")

        # Muestra vista previa
        cv2.imshow("Recolección de Datos", frame)
        
        # Condiciones de salida: ESC o completar secuencias
        if cv2.waitKey(1) & 0xFF == 27 or counter >= num_sequences:
            break

    # Libera recursos
    cap.release()
    cv2.destroyAllWindows()
    
    # Actualiza lista de gestos
    gestures = get_existing_gestures()
    print(f"\nSe recolectaron {counter} secuencias para el gesto '{gesture}'")

## CARGA DE DATOS

In [ ]:
def load_data():
    # Listas para almacenar características (X) y etiquetas (y)
    X = []
    y = []
    
    # Iterar sobre cada gesto con su índice como etiqueta
    for label_idx, gesture in enumerate(gestures):
        # Construir ruta al directorio del gesto
        gesture_dir = os.path.join(dataset_dir, gesture)
        
        # Listar solo archivos .npy válidos (30,126)
        sequences = [f for f in os.listdir(gesture_dir) if f.endswith('.npy')]
        
        # Procesar cada secuencia del gesto actual
        for seq_file in sequences:
            seq_path = os.path.join(gesture_dir, seq_file)
            sequence = np.load(seq_path)  # Cargar secuencia desde disco
            
            # Validar dimensiones del dato (30 frames × 126 landmarks)
            if sequence.shape == (sequence_length, total_landmarks):
                X.append(sequence)       # Agregar secuencia a características
                y.append(label_idx)      # Agregar índice como etiqueta
    
    # Convertir a arrays numpy y retornar con nombres de gestos
    return np.array(X), np.array(y), gestures

## ENTRENAMIENTO DEL MODELO

In [ ]:
def train_model():
    # Accede a variables globales para almacenar parámetros de normalización y lista de gestos
    global X_mean, X_std, gestures
    
    # Verifica si hay gestos registrados
    if not gestures:
        print("\nNo hay datos recolectados. Primero recolecte datos de gestos.")
        return  # Sale de la función si no hay datos

    # Mensaje de inicio de proceso
    print("\nCargando datos y preparando el entrenamiento...")
    
    # Carga datos desde el directorio
    X, y, gestures = load_data()
    
    # Convierte etiquetas a formato one-hot (ej: clase 2 → [0,0,1,0,...])
    y = tf.keras.utils.to_categorical(y)

    # Calcula media y desviación estándar para normalización
    X_mean = np.mean(X, axis=(0, 1))  # Media por landmark (ignorando tiempo y muestras)
    X_std = np.std(X, axis=(0, 1))    # Desviación estándar por landmark
    
    # Normaliza los datos (resta media, divide por desviación)
    X = (X - X_mean) / X_std

    # Define arquitectura del modelo con Keras Functional API
    inputs = tf.keras.Input(shape=(sequence_length, total_landmarks))  # Capa de entrada
    
    # Bloque CNN: Extrae patrones locales en el tiempo
    x = tf.keras.layers.Conv1D(128, 3, activation='relu', padding='same')(inputs)  # 128 filtros, kernel tamaño 3
    
    # Reducción dimensional y regularización
    x = tf.keras.layers.MaxPooling1D(2)(x)  # Reduce secuencia a la mitad
    x = tf.keras.layers.Dropout(0.5)(x)     # Apaga el 50% de neuronas aleatoriamente
    
    # Bloque LSTM: Modela dependencias temporales
    x = tf.keras.layers.LSTM(128, return_sequences=True)(x)  # LSTM con 128 unidades
    x = tf.keras.layers.Dropout(0.5)(x)                      # Nuevo dropout
    
    # Pooling global y capas densas
    x = tf.keras.layers.GlobalAveragePooling1D()(x)  # Promedia características temporales
    x = tf.keras.layers.Dense(64, activation='relu', 
                            kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)  # Regularización L2
    
    # Capa de salida con activación softmax para clasificación
    outputs = tf.keras.layers.Dense(len(gestures), activation='softmax')(x)

    # Construye el modelo completo
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Configura el proceso de entrenamiento
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # Optimizador con tasa de aprendizaje baja
        loss='categorical_crossentropy',  # Función de pérdida para clasificación múltiple
        metrics=['accuracy']              # Métrica principal a monitorear
    )

    # Entrenamiento del modelo
    print("\nIniciando entrenamiento...")
    history = model.fit(
        X, y,
        epochs=50,              # Máximo de iteraciones
        batch_size=32,          # Tamaño de lote
        validation_split=0.2,   # 20% de datos para validación
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)],  # Detención temprana
        verbose=1               # Muestra progreso
    )
    
    # Guarda el modelo entrenado
    model.save(model_path)
    print(f"\nModelo guardado en {model_path}")
    
    # Obtiene y muestra precisión final de validación
    val_accuracy = history.history['val_accuracy'][-1]  # Última época antes de detenerse
    print(f"Precisión de validación final: {val_accuracy:.2%}")

## EVALUACION DEL MODELO

In [ ]:
def evaluate():
    if not os.path.exists(model_path):
        print("\nPrimero debe entrenar el modelo.")
        return

    # Valida que existan parámetros de normalización
    if X_mean is None or X_std is None:
        print("\nERROR: Debe entrenar el modelo primero para obtener los parámetros de normalización")
        return  # Requiere normalización para consistencia en predicciones

    # Carga el modelo pre-entrenado desde disco
    model = load_model(model_path)
    print("\nCargando modelo y preparando evaluación...")

    # Inicializa buffer para almacenar secuencias temporales
    sequence = []

    # Configura captura de video desde cámara principal
    cap = cv2.VideoCapture(0)

    print("\nMostrando predicciones en tiempo real. Presiona 'ESC' para salir.")

    # Bucle principal de procesamiento en tiempo real
    while True:
        # Captura frame de la cámara
        ret, frame = cap.read()
        if not ret:
            break  # Sale si hay error en captura

        # Convierte frame a RGB para MediaPipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Detecta manos en el frame
        results = hands.process(rgb_frame)
        
        if results.multi_hand_landmarks:
            # Extrae landmarks de hasta 2 manos
            all_landmarks = []
            for hand in results.multi_hand_landmarks[:2]:
                for lm in hand.landmark:
                    all_landmarks.extend([lm.x, lm.y, lm.z])  # x,y,z normalizados
            
            # Completa con ceros si solo hay 1 mano detectada
            if len(results.multi_hand_landmarks) < 2:
                all_landmarks += [0.0] * 63  # 21 landmarks * 3 coordenadas
            
            # Almacena landmarks en el buffer temporal
            sequence.append(all_landmarks)
            
            # Dibuja landmarks en el frame
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        else:
            sequence = []  # Reinicia si no detecta manos
        
        # Mantiene solo los últimos 'sequence_length' frames
        sequence = sequence[-sequence_length:]
        
        # Realiza predicción cuando se completa una secuencia
        if len(sequence) == sequence_length:
            try:
                # Preprocesamiento y normalización
                seq_array = np.array(sequence)
                seq_array = (seq_array - X_mean) / X_std  # Normaliza
                
                # Prepara datos para el modelo (batch_size=1, sequence_length, features)
                input_data = seq_array.reshape(1, sequence_length, total_landmarks)
                
                # Predicción del modelo
                prediction = model.predict(input_data, verbose=0)[0]
                predicted_class = np.argmax(prediction)  # Clase con mayor probabilidad
                confidence = np.max(prediction)  # Confianza de la predicción
                
                # Muestra resultado si supera umbral de confianza
                if confidence > 0.8:
                    gesture = gestures[predicted_class]
                    cv2.putText(frame, f"{gesture} ({confidence:.2%})", (10, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            except Exception as e:
                print(f"\nError en predicción: {str(e)}")
                break  # Detiene ejecución ante errores críticos

        # Muestra frame con anotaciones
        cv2.imshow("Predicciones en Tiempo Real", frame)
        
        # Verifica si se presionó ESC (código ASCII 27)
        if cv2.waitKey(1) & 0xFF == 27:
            break  # Sale del bucle

    # Libera recursos de hardware
    cap.release()
    cv2.destroyAllWindows()

## REENTRENAR GESTO

In [ ]:
def retrain_gesture():
    global gestures
    if not gestures:
        print("\nNo hay gestos para reentrenar. Primero recolecte datos.")
        return

    print("\nGestos disponibles para reentrenar:")
    for i, gesture in enumerate(gestures):
        print(f"{i+1}. {gesture}")

    try:
        choice = int(input("\nSeleccione el número del gesto a reentrenar: ")) - 1
        if 0 <= choice < len(gestures):
            gesture = gestures[choice]
            gesture_dir = os.path.join(dataset_dir, gesture)
            
            for file in os.listdir(gesture_dir):
                os.remove(os.path.join(gesture_dir, file))
            
            print(f"\nDatos anteriores de '{gesture}' eliminados.")
            collect_data()
            train_model()
        else:
            print("\nSelección inválida.")
    except ValueError:
        print("\nPor favor, ingrese un número válido.")

## MENU

In [ ]:
# Menú principal
def main():
    init_system()
    
    while True:
        print("\n=== Sistema de Reconocimiento de Lenguaje de Señas ===")
        print("1. Detectar Manos")
        print("2. Recolectar Datos")
        print("3. Entrenar Modelo")
        print("4. Evaluar")
        print("5. Reentrenar Gesto")
        print("6. Salir")
        
        choice = input("\nSeleccione una opción: ")
        
        if choice == '1':
            detect_hands()
        elif choice == '2':
            collect_data()
        elif choice == '3':
            train_model()
        elif choice == '4':
            evaluate()
        elif choice == '5':
            retrain_gesture()
        elif choice == '6':
            print("\n¡Hasta luego!")
            break
        else:
            print("\nOpción inválida. Por favor, intente de nuevo.")

# MENU

In [ ]:
if __name__ == "__main__":
    main()